<a href="https://colab.research.google.com/github/GavinHacker/recsys_model/blob/master/6_recbyFM_LR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install pymysql

    100% |████████████████████████████████| 51kB 3.5MB/s 


In [0]:
import pandas as pd
import pymysql
import pymysql.cursors
from functools import reduce
import numpy as np
import pandas as pd
import uuid
import datetime
#from pyfm import pylibfm
from sklearn.feature_extraction import DictVectorizer
from sklearn.metrics.pairwise import pairwise_distances
np.set_printoptions(precision=3)
np.set_printoptions(suppress=True)
from sklearn.datasets import dump_svmlight_file
from sklearn.preprocessing import OneHotEncoder
import pickle as pkl
from sklearn.metrics import roc_auc_score, mean_squared_error
from sklearn.datasets import load_svmlight_file
from sklearn.linear_model import LogisticRegression

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!ls

drive  sample_data


In [0]:
# !wget http://www.libfm.org/libfm-1.42.src.tar.gz
!git clone https://github.com/srendle/libfm.git

Cloning into 'libfm'...
remote: Enumerating objects: 233, done.
remote: Total 233 (delta 0), reused 0 (delta 0), pack-reused 233
Receiving objects: 100% (233/233), 126.86 KiB | 1.01 MiB/s, done.
Resolving deltas: 100% (113/113), done.


In [0]:
ls

drive/  libfm/  sample_data/


In [0]:
 #!tar -xzvf libfm-1.42.src.tar.gz

In [0]:
cd libfm

/content/libfm


In [0]:
#%cd libfm-1.42.src/
#!ls
!make all

cd src/libfm; make all
make[1]: Entering directory '/content/libfm/src/libfm'
g++ -O3 -Wall -c libfm.cpp -o libfm.o
mkdir -p ../../bin/
g++ -O3 -Wall libfm.o -o ../../bin/libFM
g++ -O3 -Wall -c tools/transpose.cpp -o tools/transpose.o
mkdir -p ../../bin/
g++ -O3 tools/transpose.o -o ../../bin/transpose
g++ -O3 -Wall -c tools/convert.cpp -o tools/convert.o
mkdir -p ../../bin/
g++ -O3 tools/convert.o -o ../../bin/convert
make[1]: Leaving directory '/content/libfm/src/libfm'


In [0]:
cd bin

/content/libfm/bin


In [0]:
# /content/libfm-1.42.src/bin
#cd /content/libfm-1.42.src/bin/
ls

convert*  libFM*  transpose*


In [0]:
LIBFM_PATH = '/content/libfm/bin/libFM'
model_path = '/content/drive/My\ Drive/colabnotebook/recsysmode.fm'

In [0]:
!$LIBFM_PATH

----------------------------------------------------------------------------
libFM
  Version: 1.4.4
  Author:  Steffen Rendle, srendle@libfm.org
  WWW:     http://www.libfm.org/
This program comes with ABSOLUTELY NO WARRANTY; for details see license.txt.
This is free software, and you are welcome to redistribute it under certain
conditions; for details see license.txt.
----------------------------------------------------------------------------
-cache_size     cache size for data storage (only applicable if data is
                in binary format), default=infty
-dim            'k0,k1,k2': k0=use bias, k1=use 1-way interactions,
                k2=dim of 2-way interactions; default=1,1,8
-help           this screen
-init_stdev     stdev for initialization of 2-way factors; default=0.1
-iter           number of iterations; default=100
-learn_rate     learn_rate for SGD; default=0.1
-load_model     filename for reading the FM model
-meta           filename for meta information about dat

## FM模型训练和预测，使用libFM库蒙特卡洛法

In [0]:
def FM(train_file, test_file, classification=True, rank=10, n_iter=150):
    task = 'c' if classification else 'r'
    console_output = !$LIBFM_PATH -task $task -method mcmc -train $train_file -test $test_file -iter $n_iter -dim '1,1,$rank'  -out output_.libfm
    #console_output = !$LIBFM_PATH -task $task -method als -regular '0,0,10' -train $train_file -test $test_file -iter $n_iter -dim '1,1,$rank' -save_model recsysmode.fm -out output_.libfm
    #console_output = !$LIBFM_PATH -task $task -method sgd -train $train_file -test $test_file -iter $n_iter -dim '1,1,$rank' -save_model recsysmode.fm -out output_.libfm
    print(console_output)
    libfm_pred = pd.read_csv('output_.libfm', header=None).values.flatten()
    return libfm_pred

In [0]:
#train_file = '/content/drive/My\ Drive/colabnotebook/recsys_train.txt'
#test_file = '/content/drive/My\ Drive/colabnotebook/recsys_test.txt'

train_file = '/content/drive/My\ Drive/colabnotebook/recsys_train_scaling.txt'
test_file = '/content/drive/My\ Drive/colabnotebook/recsys_test_scaling.txt'

libfm_predict = FM(train_file, test_file, classification=False)

['----------------------------------------------------------------------------', 'libFM', '  Version: 1.4.4', '  Author:  Steffen Rendle, srendle@libfm.org', '  WWW:     http://www.libfm.org/', 'This program comes with ABSOLUTELY NO WARRANTY; for details see license.txt.', 'This is free software, and you are welcome to redistribute it under certain', 'conditions; for details see license.txt.', '----------------------------------------------------------------------------', 'Loading train...\t', 'has x = 0', 'has xt = 1', 'num_rows=280000\tnum_values=6371577\tnum_features=75619\tmin_target=1\tmax_target=5', 'Loading test... \t', 'has x = 0', 'has xt = 1', 'num_rows=9152\tnum_values=210368\tnum_features=75616\tmin_target=1\tmax_target=5', '#relations: 0', 'Loading meta data...\t', '#Iter=  0\tTrain=1.01452\tTest=0.890629', '#Iter=  1\tTrain=0.892031\tTest=0.663158', '#Iter=  2\tTrain=0.876464\tTest=0.55274', '#Iter=  3\tTrain=0.871274\tTest=0.490338', '#Iter=  4\tTrain=0.868145\tTest=0.45

In [0]:
#train_file_without_escape = '/content/drive/My Drive/colabnotebook/recsys_test.txt'
train_file_without_escape_path = '/content/drive/My Drive/colabnotebook/recsys_test_scaling.txt'

def get_test_data(path):
    data = load_svmlight_file(path)
    return data[0], data[1]

In [0]:
X, y = get_test_data(train_file_without_escape_path)

In [0]:
y

array([2., 2., 2., ..., 5., 4., 4.])

In [0]:
libfm_predict_series = pd.Series(libfm_predict)

In [0]:
libfm_predict_series_int = libfm_predict_series.apply(lambda x:int(x))

In [0]:
libfm_predict_series_int.tolist()

In [0]:
y

array([2., 2., 2., ..., 5., 4., 4.])

## 转换为评分后的测试集MSE

In [0]:
mean_squared_error(y, libfm_predict_series_int.tolist())

0.502076048951049

In [0]:
#libfm_predict_series

In [0]:
mean_squared_error(y, libfm_predict_series.tolist())

0.08685760569920237

In [0]:
# mse 0.5673076923076923

In [0]:
y[0:10]

array([2., 2., 2., 2., 2., 1., 1., 2., 2., 2.])

In [0]:
libfm_predict_series[100:500]

100    1.97529
101    1.97529
102    2.18110
103    2.18110
104    2.12399
105    2.12399
106    1.13479
107    1.13479
108    2.28783
109    2.28783
110    1.55382
111    1.55382
112    1.57852
113    1.95209
114    1.95209
115    2.11365
116    2.11365
117    1.88502
118    1.88502
119    1.92887
120    1.92887
121    2.12270
122    2.12270
123    1.83748
124    1.83748
125    1.91633
126    1.91633
127    1.96745
128    1.96745
129    1.96745
        ...   
470    1.80736
471    2.27779
472    2.27779
473    1.87991
474    1.02377
475    1.05935
476    1.76910
477    1.73278
478    1.73839
479    1.67171
480    1.96251
481    2.04542
482    1.26207
483    2.12236
484    1.64691
485    4.03943
486    4.93219
487    4.94244
488    4.94952
489    4.93831
490    4.03264
491    4.94984
492    4.92404
493    4.93621
494    4.11513
495    4.96065
496    4.94856
497    4.93610
498    4.95667
499    4.94818
Length: 400, dtype: float64

In [0]:
def get_connection():
    return pymysql.connect(host='rm-2zeqqm6994abi7b6dqo.mysql.rds.aliyuncs.com',
                               user='noone',
                               password='Huawei12#$',
                               db='recsys',
                               port=3306,
                               charset ='utf8',
                               use_unicode=True)

In [0]:
userid_collection = pd.read_sql_query("select id from system_user", get_connection())

In [0]:
#for userid in userid_collection['id'].tolist():
  #userproex_sql = 'select * from userproex_new where userid = \'%s\'' % userid
  #movie_sql = 'select * from movie where id in (select DISTINCT movieid from recmovie where userid=\'%s\')' % userid
  #movie_sql = 'select * from movie left join recmovie on movie.id=recmovie.movieid where recmovie.userid = \'%s\'' % userid
  #df_user_pro = pd.read_sql_query(userproex_sql, get_connection())
  #df_movie = pd.read_sql_query(movie_sql, get_connection())
  #df_data = pd.merge(df_movie, df_user_pro, how='left', left_on=['userid'], right_on=['userid']) 

## 对生产数据进行预测

In [0]:
sql = 'select * from recmovie left join movie on movie.id=recmovie.movieid left join userproex_new on userproex_new.userid=recmovie.userid'
df_data = pd.read_sql_query(sql, get_connection())
df_data = df_data.drop([0], axis=1)
#df_data = df_data.rename(columns = {'userid':'USERID'})
#df_data['USERID'].fillna(0)
print(df_data.columns)
df_data = df_data.loc[:,~df_data.columns.duplicated()]
#print(df_data.shape)
df_data = df_data.rename(columns = {'userid':'USERID'})
df_data['TIME_DIS'] = np.zeros(df_data.shape[0])
df_data.T

Index(['id', 'userid', 'movieid', 'rectime', 'lrrat', 'fmrat', 'srcmovieid',
       'id', 'name', 'ADD_TIME', 'type', 'actors', 'region', 'director',
       'trait', 'rat', 'enable', 'description', 'img', 'userid', 'rmax',
       'rmin', 'ravg', 'rcount', 'rsum', 'rmedian'],
      dtype='object')


,0,1,2,3,4,5,6,7,8,9,...,358,359,360,361,362,363,364,365,366,367
id,1,4,2,5,6,7,ae80cb2c-99c4-4797-9255-d5fd4a839a89,f39e0456-95dc-4d2f-8c28-4e133f0373fa,3642bc8f-4db7-4070-b42a-ba3a3e86ebc2,8a82a843-4855-4166-84bb-df1dd863f473,...,c34d35a8-311a-4104-8928-f681d21cb772,c345df00-0477-492c-a54c-d9891fdd9191,0fd2493e-fa20-4b83-be0e-11d103173ce9,162643e3-2267-4868-bf82-cc93077b8042,77ac2672-0a53-476e-b7a4-96483eb4d438,b0796180-1a2a-417f-963b-fb569ca6a3de,1f38ed3d-b0b8-4231-b63f-24e7b4d2c75d,84b9f462-2316-48d4-8246-fccc3e6686d1,6de81408-671f-44a4-8647-710c20990ca0,48fcb7d2-c6da-4b54-a2cd-cc089d1732a6
USERID,1,1,1,cf2349f9c01f9a5cd4050aebd30ab74f,cf2349f9c01f9a5cd4050aebd30ab74f,cf2349f9c01f9a5cd4050aebd30ab74f,cf2349f9c01f9a5cd4050aebd30ab74f,cf2349f9c01f9a5cd4050aebd30ab74f,cf2349f9c01f9a5cd4050aebd30ab74f,cf2349f9c01f9a5cd4050aebd30ab74f,...,cf2349f9c01f9a5cd4050aebd30ab74f,cf2349f9c01f9a5cd4050aebd30ab74f,cf2349f9c01f9a5cd4050aebd30ab74f,cf2349f9c01f9a5cd4050aebd30ab74f,cf2349f9c01f9a5cd4050aebd30ab74f,cf2349f9c01f9a5cd4050aebd30ab74f,cf2349f9c01f9a5cd4050aebd30ab74f,cf2349f9c01f9a5cd4050aebd30ab74f,cf2349f9c01f9a5cd4050aebd30ab74f,cf2349f9c01f9a5cd4050aebd30ab74f
movieid,10001432,10438426,10430817,10428476,10480973,10430817,1293143,1298574,26279449,1300077,...,4861404,1506769,26801782,6876154,25920885,1483239,1293828,25925430,1292288,25824009
rectime,2019-01-03 17:15:16,2019-01-03 17:15:16,2019-01-03 17:15:16,2019-01-03 17:15:17,2019-01-03 17:15:17,2019-01-03 17:15:17,2019-01-03 17:15:17,2019-01-03 17:15:18,2019-01-03 17:15:18,2019-01-03 17:15:18,...,2019-01-03 17:16:13,2019-01-03 17:16:13,2019-01-03 17:16:13,2019-01-03 17:16:13,2019-01-03 17:16:14,2019-01-03 17:16:14,2019-01-03 17:16:14,2019-01-03 17:16:14,2019-01-03 17:16:14,2019-01-03 17:16:14
lrrat,0.9566560968670121,0.7094158164093888,0.9246206977608482,0.9709013664018189,0.9943388169360198,0.9923293856012846,0.9892911847499388,0.9917181316565221,0.9779553256545537,0.9882757142751645,...,0.9804026653224612,0.9922989548864766,0.994322264278463,0.9871967461477341,0.9838058080674467,0.9864132321966126,0.9951047425199501,0.9907983080150248,0.9897209977315697,0.9411134313614328
fmrat,5.0,4.9844599999999994,5.0,4.013330000000001,4.01579,4.013330000000001,4.01565,4.0202800000000005,4.01428,4.03078,...,4.013330000000001,3.99511,3.9866699999999997,4.013330000000001,4.0149300000000006,4.01015,4.013330000000001,3.9866699999999997,4.00569,4.01436
srcmovieid,,,,,,,1294941,1294941,1294941,1294941,...,10727641,10727641,10727641,10727641,10727641,10727641,10727641,10727641,10727641,10727641
name,颠倒的帕特玛,浮城谜事,隐墙,小屁孩日记3,圣斗士星矢：圣域传说,隐墙,13号星期五,机器战警2,蒸发太平洋,鬼娃回魂,...,超能力者,ABC谋杀案,缉枪,国家利益,神探夏洛克：可恶的新娘,夺命枪火,007之金手指,真相,公民凯恩,脑残粉
ADD_TIME,2018-02-07 20:46:08,2018-02-07 20:26:53,2018-02-07 21:03:06,2018-02-07 20:47:00,2018-02-07 20:38:14,2018-02-07 21:03:06,2018-02-07 20:55:17,2018-02-07 20:43:08,2018-02-07 20:46:29,2018-02-07 20:41:36,...,2018-02-07 20:53:16,2018-02-07 21:03:17,2018-02-07 20:54:01,2018-02-07 20:49:14,2018-02-07 20:22:24,2018-02-07 20:37:16,2018-02-07 20:41:36,2018-02-07 20:52:29,Fri Jan 5 11:36:09 2018,2018-02-07 20:56:03
type,剧情|冒险|科幻|爱情,剧情|犯罪|悬疑|爱情,剧情|奇幻|悬疑|科幻,剧情|喜剧,奇幻|冒险|科幻|动作,剧情|奇幻|悬疑|科幻,悬疑|犯罪|惊悚|恐怖,犯罪|科幻|惊悚|动作,惊悚|剧情|犯罪|悬疑|科幻|冒险|动作,悬疑|惊悚|恐怖,...,奇幻|惊悚|剧情|犯罪|悬疑|科幻|动作,剧情|犯罪|悬疑,剧情|犯罪|悬疑|动作,惊悚|爱情|剧情|犯罪|悬疑|动作,剧情|犯罪|悬疑|惊悚,惊悚|剧情|犯罪|悬疑|冒险|动作,犯罪|冒险|惊悚|动作,剧情|犯罪|悬疑|传记|历史,剧情|犯罪|悬疑|传记,惊悚|剧情|犯罪|悬疑|喜剧|歌舞|恐怖|动作


In [0]:
print(df_data.shape)

(368, 25)


In [0]:
##df_user_pro = pd.read_sql_query("select * from userproex_new where ", get_connection())
##df_movie = pd.read_sql_query("select * from movie", get_connection())

## 需要保存向量转换器

In [0]:
v_from_pkl = None
with open('/content/drive/My Drive/colabnotebook/dict2vec', 'rb') as f:
  v_from_pkl = pkl.load(f)

In [0]:
# df_predict = pd.DataFrame(v_from_pkl.transform(predict).todense())
# df_predict[df_predict > 0].T.dropna()

In [0]:
#np.zeros(libfm_predict_series.shape[0]).shape

In [0]:
#dump_svmlight_file(testX, numpy.zeros(testX.shape[0]), '/content/drive/My Drive/colabnotebook/recsys_predict.txt')

## 保存频度统计字典

In [0]:
actors_dict = None
director_dict = None

with open('/content/drive/My Drive/colabnotebook/actors_dict', 'rb') as f:
  actors_dict = pkl.load(f)
  
with open('/content/drive/My Drive/colabnotebook/director_dict', 'rb') as f:
  director_dict = pkl.load(f)

## 与训练样本一样的方式构建预测数据的输入矩阵

In [0]:
def convert_dataframe_2_dict_list(df_main, actors_dict, director_dict):
  data_dict_list = []
  for i in df_main.index:
    _dict = {}
    is_invalid = False
    #type
    for s_type in df_main.iloc[i]['type'].split('|'):
      _dict[s_type] = 1
    #actors
    for s_actor in df_main.iloc[i]['actors'].split('|'):
      if not s_actor in actors_dict:
        print('invalid data index is ' + str(i))
        invalid_data_list.append(i)
        is_invalid = True
        break
      if actors_dict[s_actor] < 2:
        _dict['other_actor'] = 1
      else:
        _dict[s_actor] = 1
    if is_invalid == True:
      continue;
    #regios
    _dict[df_main.iloc[i]['region']] = 1
    #userid ...
    _dict[df_main.iloc[i]['USERID']] = 1
    _dict[str(df_main.iloc[i]['movieid'])] = 1
    _dict['rat'] = df_main.iloc[i]['rat']
    _dict['rmax'] = df_main.iloc[i]['rmax']
    _dict['rmin'] = df_main.iloc[i]['rmin']
    _dict['ravg'] = df_main.iloc[i]['ravg']
    _dict['rcount'] = df_main.iloc[i]['rcount']
    _dict['rsum'] = df_main.iloc[i]['rsum']
    _dict['rmedian'] = df_main.iloc[i]['rmedian']
    _dict['TIME_DIS'] = df_main.iloc[i]['TIME_DIS']
    #
    #director
    for s_director in df_main.iloc[i]['director'].split('|'):
      if director_dict[s_director] < 2:
        _dict['other_director'] = 1
      else:
        _dict[s_director] = 1
    #trait
    for s_trait in df_main.iloc[i]['trait'].split('|'):
      _dict[s_trait] = 1
    data_dict_list.append(_dict)
  return data_dict_list

In [0]:
df_data = df_data.fillna(0)

In [0]:
dict_list = convert_dataframe_2_dict_list(df_data, actors_dict, director_dict)

In [0]:
dict_list

In [0]:
predict_X = v_from_pkl.transform(dict_list)

In [0]:
#predict_X = scaler.transform(predict_X)

In [0]:
predict_file_ = '/content/drive/My Drive/colabnotebook/recsys_predict.txt'
predict_file = '/content/drive/My\ Drive/colabnotebook/recsys_predict.txt'

dump_svmlight_file(predict_X, np.zeros(predict_X.shape[0]), predict_file_)

In [0]:
libfm_predict_final = FM(train_file, predict_file, classification=False)

In [0]:
libfm_predict_final

array([5.   , 5.   , 5.   , 4.7  , 4.76 , 4.756, 4.76 , 4.76 , 4.76 ,
       4.75 , 4.758, 4.68 , 4.707, 4.756, 4.757, 4.76 , 4.737, 4.776,
       4.76 , 4.76 , 4.707, 4.755, 4.734, 4.744, 4.748, 4.76 , 4.76 ,
       4.707, 4.742, 4.76 , 4.76 , 4.749, 4.76 , 4.707, 4.707, 4.76 ,
       4.759, 4.746, 4.707, 4.76 , 4.76 , 4.76 , 4.711, 4.757, 4.76 ,
       4.804, 4.76 , 4.745, 4.76 , 4.76 , 4.744, 4.76 , 4.76 , 4.75 ,
       4.76 , 4.762, 4.76 , 4.749, 4.757, 4.756, 4.704, 4.742, 4.757,
       4.745, 4.756, 4.686, 4.76 , 4.739, 4.75 , 4.76 , 4.728, 4.744,
       4.726, 4.76 , 4.69 , 4.756, 4.707, 4.76 , 4.757, 4.736, 4.759,
       4.76 , 4.754, 4.76 , 4.76 , 4.698, 4.735, 4.756, 4.733, 4.691,
       4.76 , 4.739, 4.75 , 4.714, 4.75 , 4.739, 4.76 , 4.756, 4.76 ,
       4.716, 4.707, 4.76 , 4.753, 4.76 , 4.76 , 4.76 , 4.76 , 4.759,
       4.76 , 4.68 , 4.715, 4.801, 4.658, 4.76 , 4.759, 4.76 , 4.726,
       4.76 , 4.76 , 4.68 , 4.68 , 4.729, 4.733, 4.756, 4.752, 4.76 ,
       4.72 , 4.748,

In [0]:
df_data

,id,USERID,movieid,rectime,lrrat,fmrat,name,ADD_TIME,type,actors,...,enable,description,img,rmax,rmin,ravg,rcount,rsum,rmedian,TIME_DIS
0,1,1,10001432,2018-12-17 13:17:03,,,颠倒的帕特玛,2018-02-07 20:46:08,剧情|冒险|科幻|爱情,藤井雪代|冈本信彦|大畑伸太郎|福松进纱|加藤将之,...,1,,,0.0,0.0,0.00000,0.0,0.0,0.0,0.0
1,4,1,10438426,2018-12-17 13:19:38,,,浮城谜事,2018-02-07 20:26:53,剧情|犯罪|悬疑|爱情,郝蕾|秦昊|齐溪|朱亚文|祖锋,...,1,,,0.0,0.0,0.00000,0.0,0.0,0.0,0.0
2,2,1,10430817,2018-12-17 13:39:26,,,隐墙,2018-02-07 21:03:06,剧情|奇幻|悬疑|科幻,马蒂娜·格德克|Karlheinz Hackl|Wolfgang M|Bauer|Ulrik...,...,1,,,0.0,0.0,0.00000,0.0,0.0,0.0,0.0
3,5,cf2349f9c01f9a5cd4050aebd30ab74f,10428476,2018-12-27 16:36:07,,,小屁孩日记3,2018-02-07 20:47:00,剧情|喜剧,扎克瑞·戈登|瑞秋·哈里斯|戴文·博斯蒂克|罗伯特·卡普荣|佩顿·李斯特,...,1,,,5.0,2.0,3.85714,7.0,27.0,4.0,0.0
4,6,cf2349f9c01f9a5cd4050aebd30ab74f,10480973,2018-12-27 16:39:34,,,圣斗士星矢：圣域传说,2018-02-07 20:38:14,奇幻|冒险|科幻|动作,石川界人|赤羽根健治|小野贤章|冈本信彦|野岛健儿,...,1,,,5.0,2.0,3.85714,7.0,27.0,4.0,0.0


## 更新推荐电影recmovie表的FM预测打分或LR的点击概率

In [0]:
def update_recmovie_rat(id, rat, connection, model_type):
  rat_field = None
  if model_type == 'LR':
    rat_field = 'recmovie.lrrat'
  elif model_type == 'FM':
    rat_field = 'recmovie.fmrat'
    
  sql = 'update recmovie set %s = %s where id = %s' % (rat_field, rat, id)
  try:
      with connection.cursor() as cursor:
          cout=cursor.execute(sql)
      connection.commit()
  except:
      connection.close()

In [0]:
df_data.iloc[0]['id']

'1'

## 调用FM打分

In [0]:
def update_FM_rat(df_data, libfm_predict_final):
  index_ = 0
  for r in libfm_predict_final:
    update_recmovie_rat(df_data.iloc[index_]['id'], r, get_connection(), 'FM')
    index_ = index_ + 1

In [0]:
update_FM_rat(df_data, libfm_predict_final)

In [0]:
train_file_lr_path = '/content/drive/My Drive/colabnotebook/recsys_train_scaling_lr.txt'
test_file_lr_path = '/content/drive/My Drive/colabnotebook/recsys_test_scaling_lr.txt'

In [0]:
train_lr_path = '/content/drive/My Drive/colabnotebook/recsys_train_lr.txt'

In [0]:
train_X_lr, train_y = get_test_data(train_lr_path)

In [0]:
test_X_lr, test_y = get_test_data(test_file_lr_path)

## LR模型fit

In [0]:
lr = LogisticRegression(C=0.01,penalty='l2')
lr.fit(train_X_lr,train_y)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=0.01, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [0]:
train_X_4_scale, train_y_4_scale = get_test_data(train_lr_path)

In [0]:
train_X_4_scale.shape

(280000, 75619)

In [0]:
train_y_4_scale[train_y_4_scale == 1].shape

(159626,)

## 最终实现代码参考recsys_core

In [0]:
from sklearn import preprocessing

scaler = preprocessing.MaxAbsScaler()
scaler.fit(train_X_4_scale)

MaxAbsScaler(copy=True)

In [0]:
temp_one = v_from_pkl.transform({'1': 1,
  '10430817': 1,
  'TIME_DIS': 0.0,
  'other_actor': 1,
  'other_director': 1,
  'rat': '7.4',
  'ravg': 3.0,
  'rcount': 1.0,
  'rmax': 3.0,
  'rmedian': 3.0,
  'rmin': 3.0,
  'rsum': 3.0,
  '剧情': 1,
  '奇幻': 1,
  '德国': 1,
  '悬疑': 1,
  '文艺': 1,
  '科幻': 1,
  '马蒂娜·格德克': 1})

temp_one = v_from_pkl.transform({'1': 1,
  '10430817': 1,
  'TIME_DIS': 0.0,
  'other_actor': 1,
  'other_director': 1,
  'rat': '7.4',
  'ravg': 3.0,
  'rcount': 100.0,
  'rmax': 3.0,
  'rmedian': 3.0,
  'rmin': 3.0,
  'rsum': 3.0,
  '剧情': 1,
  '奇幻': 1,
  '德国': 1,
  '悬疑': 1,
  '文艺': 1,
  '科幻': 1,
  '马蒂娜·格德克': 1})
temp_one[temp_one != 0]

matrix([[  1.,   1.,   1.,   3., 100.,   3.,   3.,   3.,   3.,   1.,
           1.,   1.,   1.,   1.,   1.,   1.]])

In [0]:
temp_one_X = scaler.transform(temp_one)

In [0]:
temp_one_X[temp_one_X != 0]

matrix([[1.   , 1.   , 1.   , 0.6  , 0.035, 0.6  , 0.6  , 0.6  , 0.   ,
         1.   , 1.   , 1.   , 1.   , 1.   , 1.   , 1.   ]])

In [0]:
predict_X = scaler.transform(predict_X)

In [0]:
lr_predict_final = lr.predict_proba(temp_one)

In [0]:
lr_predict_final

array([[0.032, 0.968]])

## 调用更新LR点击概率

In [0]:
def update_LR_rat(df_data, lr_predict_final):
  index_ = 0
  for r in lr_predict_final:
    update_recmovie_rat(df_data.iloc[index_]['id'], r[1], get_connection(), 'LR')
    index_ = index_ + 1

In [0]:
update_LR_rat(df_data, lr_predict_final.tolist())

In [0]:
train_X_lr.shape, test_X_lr.shape

((280000, 75619), (9152, 75615))

In [0]:
X.shape

(9152, 75615)

In [0]:
predict_X

<5x75619 sparse matrix of type '<class 'numpy.float64'>'
	with 94 stored elements in Compressed Sparse Row format>

In [0]:
lr.classes_

array([0., 1.])

In [0]:
lr_predict_final.tolist()

[[0.9976049939496754, 0.0023950060503245886],
 [0.9998143586577533, 0.00018564134224680022],
 [0.9983921456376658, 0.0016078543623342142],
 [0.0, 1.0],
 [0.0, 1.0]]